In [2]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.utils.model_zoo as model_zoo
import numpy as np

import torch
import torch.nn as nn
import torch.nn.init as init

from torch.nn import functional, init

import transformer, layer

In [3]:
x = torch.rand(1, 3, 256,256)
x.shape

torch.Size([1, 3, 256, 256])

In [4]:
#Feature extractor (depth 4) output
conv1 = nn.Conv2d(3,3,1,1,0)
reflecPad1 = nn.ReflectionPad2d((1,1,1,1))
        # 226 x 226

conv2 = nn.Conv2d(3,64,3,1,0)
relu2 = nn.ReLU(inplace=True)
        # 224 x 224

reflecPad3 = nn.ReflectionPad2d((1,1,1,1))
conv3 = nn.Conv2d(64,64,3,1,0)
relu3 = nn.ReLU(inplace=True)
        # 224 x 224

maxPool = nn.MaxPool2d(kernel_size=2,stride=2,return_indices = True)
        # 112 x 112

reflecPad4 = nn.ReflectionPad2d((1,1,1,1))
conv4 = nn.Conv2d(64,128,3,1,0)
relu4 = nn.ReLU(inplace=True)
        # 112 x 112

reflecPad5 = nn.ReflectionPad2d((1,1,1,1))
conv5 = nn.Conv2d(128,128,3,1,0)
relu5 = nn.ReLU(inplace=True)
        # 112 x 112

maxPool2 = nn.MaxPool2d(kernel_size=2,stride=2,return_indices = True)
        # 56 x 56

reflecPad6 = nn.ReflectionPad2d((1,1,1,1))
conv6 = nn.Conv2d(128,256,3,1,0)
relu6 = nn.ReLU(inplace=True)
        # 56 x 56

reflecPad7 = nn.ReflectionPad2d((1,1,1,1))
conv7 = nn.Conv2d(256,256,3,1,0)
relu7 = nn.ReLU(inplace=True)
        # 56 x 56

reflecPad8 = nn.ReflectionPad2d((1,1,1,1))
conv8 = nn.Conv2d(256,256,3,1,0)
relu8 = nn.ReLU(inplace=True)
        # 56 x 56

reflecPad9 = nn.ReflectionPad2d((1,1,1,1))
conv9 = nn.Conv2d(256,256,3,1,0)
relu9 = nn.ReLU(inplace=True)
        # 56 x 56

maxPool3 = nn.MaxPool2d(kernel_size=2,stride=2,return_indices = True)
        # 28 x 28

reflecPad10 = nn.ReflectionPad2d((1,1,1,1))
conv10 = nn.Conv2d(256,512,3,1,0)
relu10 = nn.ReLU(inplace=True)


out = conv1(x)
out = reflecPad1(out)
out = conv2(out)
out = relu2(out)
out = reflecPad3(out)
out = conv3(out)
pool1 = relu3(out)
out,pool_idx = maxPool(pool1)

out = reflecPad4(out)
out = conv4(out)
z1 = out = relu4(out)
out = reflecPad5(out)
out = conv5(out)
pool2 = relu5(out)
out,pool_idx2 = maxPool2(pool2)

out = reflecPad6(out)
out = conv6(out)
out = relu6(out)
out = reflecPad7(out)
out = conv7(out)
out = relu7(out)
out = reflecPad8(out)
out = conv8(out)
z2 = out = relu8(out)
out = reflecPad9(out)
out = conv9(out)
pool3 = relu9(out)
out,pool_idx3 = maxPool3(pool3)

out = reflecPad10(out)
out = conv10(out)
out = relu10(out)
z8 = out
out.shape

torch.Size([1, 512, 32, 32])

In [5]:
fan_s8 = layer.FeatureAwareNorm(128, 512, scale=4)
fan_s4 = layer.FeatureAwareNorm(128, 256, scale=2)
fan_s2 = layer.FeatureAwareNorm(128, 128, scale=1)

In [6]:
tconv1 = nn.Conv2d(3,3,1,1,0)
treflecPad1 = nn.ReflectionPad2d((1,1,1,1))
        # 226 x 226

tconv2 = nn.Conv2d(3,64,3,1,0)
trelu2 = nn.ReLU(inplace=True)
        # 224 x 224

treflecPad3 = nn.ReflectionPad2d((1,1,1,1))
tconv3 = nn.Conv2d(64,64,3,1,0)
trelu3 = nn.ReLU(inplace=True)
        # 224 x 224

tmaxPool = nn.MaxPool2d(kernel_size=2,stride=2,return_indices = True)
        # 112 x 112

treflecPad4 = nn.ReflectionPad2d((1,1,1,1))
tconv4 = nn.Conv2d(64,128,3,1,0)
trelu4 = nn.ReLU(inplace=True)

tout = tconv1(x)
tout = treflecPad1(tout)
tout = tconv2(tout)
tout = trelu2(tout)
tout = treflecPad3(tout)
tout = tconv3(tout)
tpool = trelu3(tout)
tout,tpool_idx = tmaxPool(tpool)
tout = treflecPad4(tout)
tout = tconv4(tout)
tout = trelu4(tout)
print(tout.shape)

torch.Size([1, 128, 128, 128])


In [7]:
y = fan_s8(tout, z8)
y = fan_s4(y, z2)
y = fan_s2(y, z1)

C:\Users\Hyuna\anaconda3\lib\site-packages\torch\nn\functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [8]:
ireflecPad5 = nn.ReflectionPad2d((1,1,1,1))
iconv5 = nn.Conv2d(128,64,3,1,0)
irelu5 = nn.ReLU(inplace=True)
        # 112 x 112

iunpool = nn.UpsamplingNearest2d(scale_factor=2)
        # 224 x 224

ireflecPad6 = nn.ReflectionPad2d((1,1,1,1))
iconv6 = nn.Conv2d(64,64,3,1,0)
irelu6 = nn.ReLU(inplace=True)
        # 224 x 224

ireflecPad7 = nn.ReflectionPad2d((1,1,1,1))
iconv7 = nn.Conv2d(64,3,3,1,0)

iout = ireflecPad5(y+tout)
iout = iconv5(iout)
iout = irelu5(iout)
iout = iunpool(iout)
iout = ireflecPad6(iout)
iout = iconv6(iout)
iout = irelu6(iout)
iout = ireflecPad7(iout)
iout = iconv7(iout)

print(iout.shape)

torch.Size([1, 3, 256, 256])
